In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [3]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [4]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [5]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [6]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = v[v['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [7]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    
    y['MEAN_ALL'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thAll'] = epoct
    y['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    y = np.round(y,3)
    return y

In [8]:
def IndecesALLAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_AM'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_AM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85thALL_AM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_AM'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [9]:
def IndecesALLMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_MD'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_MD'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thAll_MD'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85thAll_MD'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_MD'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [10]:
def IndecesALLPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_SL_PM'] = y['SPDLIMIT'].div(y['MEAN_ALL'])
    z['PTI_SL_PM'] = y['SPDLIMIT'].div(y['PcntSpeed_5thAll'])
    z['TTI_85thAll_PM'] = y['PcntSpeed_85thAll'].div(y['MEAN_ALL'])
    z['PTI_85thAll_PM'] = y['PcntSpeed_85thAll'].div(y['PcntSpeed_5thAll'])
    z['BTI_PM'] = (y['MEAN_ALL'].div(y['PcntSpeed_5thAll']))-1
    z = np.round(z,3)
    return z

In [11]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_Wkd'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)

    
    y['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWd'] = epoct
    y['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [12]:
def IndecesWdAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_AM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [13]:
def IndecesWdMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_MD'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [14]:
def IndecesWdPM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['MEAN_Wkd'])
    z['PTI_85thWd_PM'] = y['PcntSpeed_85thWd'].div(y['PcntSpeed_5thWd'])
    z = np.round(z,3)
    return z

In [15]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WEnd'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_85thWend'] = epoct
    y['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [16]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_Wdtime'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    y['PcntSpeed_5thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_Wdtime'] = epoct
    y['PcntInterval_Wdtime'] = ((epoct/(168*len(numberdays))*100))#168 represents epochs between 6AM and 8PM
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [17]:
def IndecesWDtimeAM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_AM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_AM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [18]:
def IndecesWDtimeMD(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_MD'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_MD'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [19]:
def IndecesWDtimePM(y):
    z = pd.DataFrame(columns = [])
    z['TTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_60thWdtime_PM'] = y['PcntSpeed_60thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z['TTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['MEAN_Wdtime'])
    z['PTI_85thWdtime_PM'] = y['PcntSpeed_85thWdtime'].div(y['PcntSpeed_5thWdtime'])
    z = np.round(z,3)
    return z

In [20]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdAM'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayAM'] = y['MEAN_WkdAM']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayAM'] = epoct
    y['PcntInterval_WdayAM'] = ((epoct/(36*len(numberdays))*100))#36 represents epochs between 6AM and 9AM
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [21]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdMD'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayMD'] = y['MEAN_WkdMD']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayMD'] = epoct
    y['PcntInterval_WdayMD'] = ((epoct/(72*len(numberdays))*100))#72 represents epochs between 9AM and 3PM
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [22]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN_WkdPM'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    y['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    y['AvgSpeed_WdayPM'] = y['MEAN_WkdPM']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    y['TotalIntervals_WdayPM'] = epoct
    y['PcntInterval_WdayPM'] = ((epoct/(36*len(numberdays))*100))#36 represents epochs between 3PM and 6PM
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    x['VARIANCE'] = x['STDDEV']**2
    Var2 = (x['VARIANCE'].groupby(x['LINK_DIR']))
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    y['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    y = np.round(y,3)
    return y

In [23]:
#For all days and all time periods
All = timeframe2(herepath,0,288,False,Both)
bylink = HEREstatsALL(All)
bylink = bylink.drop(['SPDLIMIT'], axis = 1)
All = pd.DataFrame()
bylink.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll
LINK_DIR,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445
107030439T,35.148,44.734,17.396,7857,7.474,9.933
107030440T,34.863,45.355,17.086,7325,6.968,10.308
107030444F,36.180,47.219,16.154,9311,8.857,11.217
107030445F,36.602,47.840,16.775,9256,8.805,11.256


In [24]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd = SpeedWd.drop(['SPDLIMIT'], axis = 1)
Wkd = pd.DataFrame()
SpeedWd.head()

,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,,,
107030432T,38.331,47.840,19.260,4967,6.790,10.330
107030439T,35.042,44.734,17.396,6701,9.160,9.946
107030440T,34.961,45.355,17.396,6266,8.566,10.274
107030444F,35.891,47.219,15.532,7948,10.865,11.311
107030445F,36.193,47.840,16.775,7926,10.835,11.290


In [25]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend = SpeedWend.drop(['SPDLIMIT'], axis = 1)
Wend = pd.DataFrame()
SpeedWend.head()

,MEAN_WEnd,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,,
107030432T,40.127,50.325,723,2.414,10.989
107030439T,35.565,45.355,1082,3.612,9.878
107030440T,34.040,45.355,986,3.292,10.543
107030444F,37.839,47.840,1266,4.227,10.472
107030445F,38.929,49.083,1248,4.167,10.765


In [ ]:
y = pd.DataFrame()
Wend = timeframe2(herepath,0,288,True,Weekend)
y['VAR1'] = (Wend['MEAN']).groupby(Wend['LINK_DIR']).mean()
Wend['VARIANCE'] = Wend['STDDEV']**2
#y['VAR2'] = (Wend['VARIANCE'].groupby(Wend['LINK_DIR']))
y
#Var = (Var1.var(ddof=1))+(Var2.sum()/len(Wend))
#y['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)

In [26]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime = SpeedWdtime.drop(['SPDLIMIT'], axis = 1)
Wdtime = pd.DataFrame()
SpeedWdtime.head()

,MEAN_Wdtime,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,,,
107030432T,37.981,41.627,47.840,18.639,4408,10.330,10.395
107030439T,34.857,38.521,44.734,17.396,6168,14.454,9.998
107030440T,34.692,37.899,45.355,16.775,5827,13.655,10.279
107030444F,35.526,40.074,46.908,14.911,7452,17.463,11.364
107030445F,35.837,40.384,47.219,16.154,7288,17.079,11.294


In [27]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM = SpeedWdayAM.drop(['SPDLIMIT'], axis = 1)
WdtimeAM = pd.DataFrame()
SpeedWdayAM.head()

,MEAN_WkdAM,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,,
107030432T,38.721,20.503,38.721,728,7.962,10.574
107030439T,33.945,16.775,33.945,977,10.685,10.658
107030440T,34.521,16.775,34.521,841,9.197,10.970
107030444F,33.329,14.911,33.329,1299,14.206,11.937
107030445F,33.237,15.532,33.237,1333,14.578,11.451


In [28]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD = SpeedWdayMD.drop(['SPDLIMIT'], axis = 1)
WdtimeMD = pd.DataFrame()
SpeedWdayMD.head()

,MEAN_WkdMD,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,,
107030432T,37.987,18.950,37.987,2145,11.729,10.323
107030439T,35.178,18.639,35.178,3254,17.793,9.530
107030440T,34.832,17.396,34.832,3126,17.093,9.954
107030444F,36.990,16.775,36.990,3859,21.101,10.767
107030445F,37.058,17.396,37.058,3796,20.757,10.935


In [29]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimePM)
SpeedWdayPM = SpeedWdayPM.drop(['SPDLIMIT'], axis = 1)
WdtimePM = pd.DataFrame()
SpeedWdayPM.head()

,MEAN_WkdPM,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,,
107030432T,36.978,18.639,36.978,1271,13.900,10.072
107030439T,34.551,15.532,34.551,1511,16.524,10.237
107030440T,34.414,16.154,34.414,1491,16.306,10.294
107030444F,33.650,11.805,33.650,1808,19.773,11.716
107030445F,34.487,14.290,34.487,1738,19.007,11.575


In [30]:
#Calculates AM Indeces for all 365 days for all links
AllAM = timeframe2(herepath,72,108,False,Both)
HstatsAM = HEREstatsALL(AllAM)
IndecesAllAM = IndecesALLAM(HstatsAM)
AllAM = pd.DataFrame()
IndecesAllAM.head()

,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM
LINK_DIR,,,,,
107030432T,1.276,2.581,1.244,2.516,1.023
107030439T,1.309,2.667,1.327,2.704,1.037
107030440T,1.293,2.769,1.329,2.846,1.142
107030444F,1.325,3.000,1.398,3.167,1.265
107030445F,1.331,2.880,1.387,3.000,1.163


In [31]:
#Calculates MD Indeces for all 365 days for all links
AllMD = timeframe2(herepath,108,180,False,Both)
HstatsMD = HEREstatsALL(AllMD)
IndecesAllMD = IndecesALLMD(HstatsMD)
AllMD = pd.DataFrame()
IndecesAllMD.head()

,TTI_SL_MD,PTI_SL_MD,TTI_85thAll_MD,PTI_85thAll_MD,BTI_MD
LINK_DIR,,,,,
107030432T,1.295,2.581,1.247,2.484,0.992
107030439T,1.272,2.400,1.272,2.400,0.887
107030440T,1.293,2.572,1.293,2.572,0.989
107030444F,1.209,2.618,1.285,2.782,1.165
107030445F,1.200,2.483,1.284,2.655,1.069


In [32]:
#Calculates PM Indeces for all 365 days for all links
AllPM = timeframe2(herepath,180,216,False,Both)
HstatsPM = HEREstatsALL(AllPM)
IndecesAllPM = IndecesALLPM(HstatsPM)
AllPM = pd.DataFrame()
IndecesAllPM.head()

,TTI_SL_PM,PTI_SL_PM,TTI_85thAll_PM,PTI_85thAll_PM,BTI_PM
LINK_DIR,,,,,
107030432T,1.334,2.667,1.267,2.533,0.999
107030439T,1.291,2.880,1.291,2.880,1.231
107030440T,1.302,2.880,1.302,2.880,1.212
107030444F,1.307,3.600,1.343,3.700,1.754
107030445F,1.272,3.000,1.343,3.167,1.358


In [33]:
#Calculates weekday non-holiday indeces for AM for all links
WdAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdAM = HEREstatsWd(WdAM)
IndecesWdayAM = IndecesWdAM(HstatsWdAM)
WdAM = pd.DataFrame()
IndecesWdayAM.head()

,TTI_85thWd_AM,PTI_85thWd_AM
LINK_DIR,,
107030432T,1.252,2.364
107030439T,1.336,2.704
107030440T,1.332,2.741
107030444F,1.398,3.125
107030445F,1.383,2.960


In [34]:
#Calculates weekday non-holiday indeces for MD for all links
WdMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdMD = HEREstatsWd(WdMD)
IndecesWdayMD = IndecesWdMD(HstatsWdMD)
WdMD = pd.DataFrame()
IndecesWdayMD.head()

,TTI_85thWd_MD,PTI_85thWd_MD
LINK_DIR,,
107030432T,1.259,2.525
107030439T,1.272,2.400
107030440T,1.284,2.572
107030444F,1.293,2.852
107030445F,1.291,2.750


In [35]:
#Calculates weekday non-holiday indeces for PM for all links
WdPM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdPM = HEREstatsWd(WdPM)
IndecesWdayPM = IndecesWdPM(HstatsWdPM)
WdPM = pd.DataFrame()
IndecesWdayPM.head()

,TTI_85thWd_PM,PTI_85thWd_PM
LINK_DIR,,
107030432T,1.269,2.517
107030439T,1.295,2.880
107030440T,1.300,2.769
107030444F,1.348,3.842
107030445F,1.351,3.261


In [36]:
#Calculates daytime indeces for AM for all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
HstatsWdtimeAM = HEREstatsWdtime(WdtimeAM)
IndecesWdtimeAM = IndecesWDtimeAM(HstatsWdtimeAM)
WdtimeAM = pd.DataFrame()
IndecesWdtimeAM.head()

,TTI_60thWdtime_AM,PTI_60thWdtime_AM,TTI_85thWdtime_AM,PTI_85thWdtime_AM
LINK_DIR,,,,
107030432T,1.091,2.061,1.252,2.364
107030439T,1.107,2.241,1.336,2.704
107030440T,1.134,2.333,1.332,2.741
107030444F,1.109,2.479,1.398,3.125
107030445F,1.084,2.320,1.383,2.960


In [37]:
#Calculates daytime indeces for MD for all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
HstatsWdtimeMD = HEREstatsWdtime(WdtimeMD)
IndecesWdtimeMD = IndecesWDtimeMD(HstatsWdtimeMD)
WdtimeMD = pd.DataFrame()
IndecesWdtimeMD.head()

,TTI_60thWdtime_MD,PTI_60thWdtime_MD,TTI_85thWdtime_MD,PTI_85thWdtime_MD
LINK_DIR,,,,
107030432T,1.096,2.197,1.259,2.525
107030439T,1.095,2.067,1.272,2.400
107030440T,1.088,2.179,1.284,2.572
107030444F,1.125,2.481,1.293,2.852
107030445F,1.123,2.393,1.291,2.750


In [38]:
#Calculates daytime indeces for PM for all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
HstatsWdtimePM = HEREstatsWdtime(WdtimePM)
IndecesWdtimePM = IndecesWDtimePM(HstatsWdtimePM)
WdtimePM = pd.DataFrame()
IndecesWdtimePM.head()

,TTI_60thWdtime_PM,PTI_60thWdtime_PM,TTI_85thWdtime_PM,PTI_85thWdtime_PM
LINK_DIR,,,,
107030432T,1.092,2.167,1.269,2.517
107030439T,1.115,2.480,1.295,2.880
107030440T,1.083,2.308,1.300,2.769
107030444F,1.145,3.263,1.348,3.842
107030445F,1.135,2.739,1.351,3.261


In [39]:
master = bylink

In [40]:
master = pd.merge(master, SpeedWd, left_index = True, right_index = True)
SpeedWd = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,6.790,10.330
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,9.160,9.946
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,8.566,10.274
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,10.865,11.311
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,10.835,11.290


In [41]:
master = pd.merge(master, SpeedWend, left_index = True, right_index = True)
SpeedWend = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd,MEAN_WEnd,PcntSpeed_85thWend,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend
LINK_DIR,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,6.790,10.330,40.127,50.325,723,2.414,10.989
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,9.160,9.946,35.565,45.355,1082,3.612,9.878
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,8.566,10.274,34.040,45.355,986,3.292,10.543
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,10.865,11.311,37.839,47.840,1266,4.227,10.472
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,10.835,11.290,38.929,49.083,1248,4.167,10.765


In [42]:
master = pd.merge(master, SpeedWdtime, left_index = True, right_index = True)
SpeedWdtime = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TotalIntervals_85thWend,PcntInterval_85thWend,StdDev_85thWend,MEAN_Wdtime,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,723,2.414,10.989,37.981,41.627,47.840,18.639,4408,10.330,10.395
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1082,3.612,9.878,34.857,38.521,44.734,17.396,6168,14.454,9.998
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,986,3.292,10.543,34.692,37.899,45.355,16.775,5827,13.655,10.279
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1266,4.227,10.472,35.526,40.074,46.908,14.911,7452,17.463,11.364
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1248,4.167,10.765,35.837,40.384,47.219,16.154,7288,17.079,11.294


In [43]:
master = pd.merge(master, SpeedWdayAM, left_index = True, right_index = True)
SpeedWdayAM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,PcntSpeed_5thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime,MEAN_WkdAM,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,18.639,4408,10.330,10.395,38.721,20.503,38.721,728,7.962,10.574
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,17.396,6168,14.454,9.998,33.945,16.775,33.945,977,10.685,10.658
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,16.775,5827,13.655,10.279,34.521,16.775,34.521,841,9.197,10.970
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,14.911,7452,17.463,11.364,33.329,14.911,33.329,1299,14.206,11.937
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,16.154,7288,17.079,11.294,33.237,15.532,33.237,1333,14.578,11.451


In [44]:
master = pd.merge(master, SpeedWdayMD, left_index = True, right_index = True)
SpeedWdayMD = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM,MEAN_WkdMD,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,38.721,728,7.962,10.574,37.987,18.950,37.987,2145,11.729,10.323
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,33.945,977,10.685,10.658,35.178,18.639,35.178,3254,17.793,9.530
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,34.521,841,9.197,10.970,34.832,17.396,34.832,3126,17.093,9.954
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,33.329,1299,14.206,11.937,36.990,16.775,36.990,3859,21.101,10.767
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,33.237,1333,14.578,11.451,37.058,17.396,37.058,3796,20.757,10.935


In [45]:
master = pd.merge(master, SpeedWdayPM, left_index = True, right_index = True)
SpeedWdayPM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD,MEAN_WkdPM,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,37.987,2145,11.729,10.323,36.978,18.639,36.978,1271,13.900,10.072
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,35.178,3254,17.793,9.530,34.551,15.532,34.551,1511,16.524,10.237
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,34.832,3126,17.093,9.954,34.414,16.154,34.414,1491,16.306,10.294
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,36.990,3859,21.101,10.767,33.650,11.805,33.650,1808,19.773,11.716
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,37.058,3796,20.757,10.935,34.487,14.290,34.487,1738,19.007,11.575


In [46]:
master = pd.merge(master, IndecesAllAM, left_index = True, right_index = True)
IndecesAllAM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,18.639,36.978,1271,13.900,10.072,1.276,2.581,1.244,2.516,1.023
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,15.532,34.551,1511,16.524,10.237,1.309,2.667,1.327,2.704,1.037
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,16.154,34.414,1491,16.306,10.294,1.293,2.769,1.329,2.846,1.142
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,11.805,33.650,1808,19.773,11.716,1.325,3.000,1.398,3.167,1.265
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,14.290,34.487,1738,19.007,11.575,1.331,2.880,1.387,3.000,1.163


In [47]:
master = pd.merge(master, IndecesAllMD, left_index = True, right_index = True)
IndecesAllMD = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_SL_AM,PTI_SL_AM,TTI_85thALL_AM,PTI_85thALL_AM,BTI_AM,TTI_SL_MD,PTI_SL_MD,TTI_85thAll_MD,PTI_85thAll_MD,BTI_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.276,2.581,1.244,2.516,1.023,1.295,2.581,1.247,2.484,0.992
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.309,2.667,1.327,2.704,1.037,1.272,2.400,1.272,2.400,0.887
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.293,2.769,1.329,2.846,1.142,1.293,2.572,1.293,2.572,0.989
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.325,3.000,1.398,3.167,1.265,1.209,2.618,1.285,2.782,1.165
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.331,2.880,1.387,3.000,1.163,1.200,2.483,1.284,2.655,1.069


In [48]:
master = pd.merge(master, IndecesAllPM, left_index = True, right_index = True)
IndecesAllPM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_SL_MD,PTI_SL_MD,TTI_85thAll_MD,PTI_85thAll_MD,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85thAll_PM,PTI_85thAll_PM,BTI_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.295,2.581,1.247,2.484,0.992,1.334,2.667,1.267,2.533,0.999
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.272,2.400,1.272,2.400,0.887,1.291,2.880,1.291,2.880,1.231
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.293,2.572,1.293,2.572,0.989,1.302,2.880,1.302,2.880,1.212
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.209,2.618,1.285,2.782,1.165,1.307,3.600,1.343,3.700,1.754
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.200,2.483,1.284,2.655,1.069,1.272,3.000,1.343,3.167,1.358


In [49]:
master = pd.merge(master, IndecesWdayAM, left_index = True, right_index = True)
IndecesWdayAM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_85thAll_MD,PTI_85thAll_MD,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85thAll_PM,PTI_85thAll_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.247,2.484,0.992,1.334,2.667,1.267,2.533,0.999,1.252,2.364
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.272,2.400,0.887,1.291,2.880,1.291,2.880,1.231,1.336,2.704
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.293,2.572,0.989,1.302,2.880,1.302,2.880,1.212,1.332,2.741
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.285,2.782,1.165,1.307,3.600,1.343,3.700,1.754,1.398,3.125
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.284,2.655,1.069,1.272,3.000,1.343,3.167,1.358,1.383,2.960


In [50]:
master = pd.merge(master, IndecesWdayMD, left_index = True, right_index = True)
IndecesWdayMD = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,BTI_MD,TTI_SL_PM,PTI_SL_PM,TTI_85thAll_PM,PTI_85thAll_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,0.992,1.334,2.667,1.267,2.533,0.999,1.252,2.364,1.259,2.525
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,0.887,1.291,2.880,1.291,2.880,1.231,1.336,2.704,1.272,2.400
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,0.989,1.302,2.880,1.302,2.880,1.212,1.332,2.741,1.284,2.572
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.165,1.307,3.600,1.343,3.700,1.754,1.398,3.125,1.293,2.852
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.069,1.272,3.000,1.343,3.167,1.358,1.383,2.960,1.291,2.750


In [51]:
master = pd.merge(master, IndecesWdayPM, left_index = True, right_index = True)
IndecesWdayPM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,PTI_SL_PM,TTI_85thAll_PM,PTI_85thAll_PM,BTI_PM,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD,TTI_85thWd_PM,PTI_85thWd_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,2.667,1.267,2.533,0.999,1.252,2.364,1.259,2.525,1.269,2.517
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,2.880,1.291,2.880,1.231,1.336,2.704,1.272,2.400,1.295,2.880
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,2.880,1.302,2.880,1.212,1.332,2.741,1.284,2.572,1.300,2.769
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,3.600,1.343,3.700,1.754,1.398,3.125,1.293,2.852,1.348,3.842
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,3.000,1.343,3.167,1.358,1.383,2.960,1.291,2.750,1.351,3.261


In [52]:
master = pd.merge(master, IndecesWdtimeAM, left_index = True, right_index = True)
IndecesWdtimeAM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_85thWd_AM,PTI_85thWd_AM,TTI_85thWd_MD,PTI_85thWd_MD,TTI_85thWd_PM,PTI_85thWd_PM,TTI_60thWdtime_AM,PTI_60thWdtime_AM,TTI_85thWdtime_AM,PTI_85thWdtime_AM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.252,2.364,1.259,2.525,1.269,2.517,1.091,2.061,1.252,2.364
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.336,2.704,1.272,2.400,1.295,2.880,1.107,2.241,1.336,2.704
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.332,2.741,1.284,2.572,1.300,2.769,1.134,2.333,1.332,2.741
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.398,3.125,1.293,2.852,1.348,3.842,1.109,2.479,1.398,3.125
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.383,2.960,1.291,2.750,1.351,3.261,1.084,2.320,1.383,2.960


In [53]:
master = pd.merge(master, IndecesWdtimeMD, left_index = True, right_index = True)
IndecesWdtimeMD = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_85thWd_PM,PTI_85thWd_PM,TTI_60thWdtime_AM,PTI_60thWdtime_AM,TTI_85thWdtime_AM,PTI_85thWdtime_AM,TTI_60thWdtime_MD,PTI_60thWdtime_MD,TTI_85thWdtime_MD,PTI_85thWdtime_MD
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.269,2.517,1.091,2.061,1.252,2.364,1.096,2.197,1.259,2.525
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.295,2.880,1.107,2.241,1.336,2.704,1.095,2.067,1.272,2.400
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.300,2.769,1.134,2.333,1.332,2.741,1.088,2.179,1.284,2.572
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.348,3.842,1.109,2.479,1.398,3.125,1.125,2.481,1.293,2.852
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.351,3.261,1.084,2.320,1.383,2.960,1.123,2.393,1.291,2.750


In [54]:
master = pd.merge(master, IndecesWdtimePM, left_index = True, right_index = True)
IndecesWdtimePM = pd.DataFrame()
master.head()

,MEAN_ALL,PcntSpeed_85thAll,PcntSpeed_5thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll,MEAN_Wkd,PcntSpeed_85thWd,PcntSpeed_5thWd,TotalIntervals_85thWd,...,TTI_85thWdtime_AM,PTI_85thWdtime_AM,TTI_60thWdtime_MD,PTI_60thWdtime_MD,TTI_85thWdtime_MD,PTI_85thWdtime_MD,TTI_60thWdtime_PM,PTI_60thWdtime_PM,TTI_85thWdtime_PM,PTI_85thWdtime_PM
LINK_DIR,,,,,,,,,,,,,,,,,,,,,
107030432T,38.553,48.461,19.260,5746,5.466,10.445,38.331,47.840,19.260,4967,...,1.252,2.364,1.096,2.197,1.259,2.525,1.092,2.167,1.269,2.517
107030439T,35.148,44.734,17.396,7857,7.474,9.933,35.042,44.734,17.396,6701,...,1.336,2.704,1.095,2.067,1.272,2.400,1.115,2.480,1.295,2.880
107030440T,34.863,45.355,17.086,7325,6.968,10.308,34.961,45.355,17.396,6266,...,1.332,2.741,1.088,2.179,1.284,2.572,1.083,2.308,1.300,2.769
107030444F,36.180,47.219,16.154,9311,8.857,11.217,35.891,47.219,15.532,7948,...,1.398,3.125,1.125,2.481,1.293,2.852,1.145,3.263,1.348,3.842
107030445F,36.602,47.840,16.775,9256,8.805,11.256,36.193,47.840,16.775,7926,...,1.383,2.960,1.123,2.393,1.291,2.750,1.135,2.739,1.351,3.261


In [55]:
master.to_csv('HEREstatsWMEAN.csv')